In [15]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

INPUT_PATH = Path('../input')
OUTPUT_PATH = Path('../output')
pd.set_option('display.float_format', str)

In [46]:
%%time

tran = pd.read_csv(INPUT_PATH / 'trxn.csv')
aum = pd.read_csv(INPUT_PATH / 'aum.csv')
balance = pd.read_csv(INPUT_PATH / 'balance.csv')
client = pd.read_csv(INPUT_PATH / 'client.csv')
campaigns = pd.read_csv(INPUT_PATH / 'com.csv')
deals = pd.read_csv(INPUT_PATH / 'deals.csv')
dict_merchant_category_code = pd.read_csv(INPUT_PATH / 'dict_mcc.csv')
payments = pd.read_csv(INPUT_PATH / 'payments.csv')
funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
appl = pd.read_csv(INPUT_PATH / 'appl.csv')

/opt/conda/lib/python3.7/site-packages/decorator.py:231: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


CPU times: user 6.99 s, sys: 647 ms, total: 7.64 s
Wall time: 7.64 s


In [33]:
def show_nan_number(df):
    for col in df.columns:
        print(f'NaNs for "{col}": {df[col].isna().sum()}')

## Table look up

In [20]:
# card operations
print(tran.shape)
tran.head()

(3035705, 11)


,client_id,card_id,tran_time,tran_amt_rur,mcc_cd,merchant_cd,txn_country,txn_city,tsp_name,txn_comment_1,txn_comment_2
0,-1893104556496814867,-6909844117230568380,2019-07-01 00:00:00,6071.0,6011.0,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN
1,-1893104556496814867,-6909844117230568380,2019-07-29 00:00:00,6071.0,6011.0,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN
2,-1893104556496814867,-6909844117230568380,2019-08-31 00:00:00,7285.0,6011.0,3796217713062573289,RUS,N. NOVGOROD,MOLODEZHNYY PR. DEPO,Cash withdrawal through an ATM,NaN
3,6886062013213911831,6527755410945490683,2019-08-14 10:38:33,2428.0,6011.0,-774579209084196290,RUS,RADUZHNYI,TTS SITI,Cash withdrawal through an ATM,NaN
4,6886062013213911831,6527755410945490683,2019-07-17 12:27:58,6071.0,6011.0,-7735007003326589238,RUS,RADUZHNYI,ODTS AGANGRAD,Cash withdrawal through an ATM,NaN


In [34]:
show_info(tran)

NaNs for "client_id": 0
NaNs for "card_id": 0
NaNs for "tran_time": 0
NaNs for "tran_amt_rur": 0
NaNs for "mcc_cd": 132
NaNs for "merchant_cd": 0
NaNs for "txn_country": 0
NaNs for "txn_city": 5385
NaNs for "tsp_name": 2162
NaNs for "txn_comment_1": 0
NaNs for "txn_comment_2": 2638786


In [21]:
tran['mcc_cd'].nunique()

307

In [22]:
tran['tran_amt_rur'].describe()

count            3035705.0
mean     2744.739667391924
std     15011.821343982692
min                    0.0
25%                  194.0
50%                  486.0
75%                 1275.0
max              7527563.0
Name: tran_amt_rur, dtype: float64

In [25]:
tran['tsp_name'].sample(20)

785265           MAGNIT GM ISHIM
998643       SEMYA NA SOLDATOVA.
282047         KHIMCHISTKA NIKKO
329737               PYATEROCHKA
462088                 DNS ORION
6650                         ATM
194251        UNIQLO EVROPEYSKIY
2523178                  MONETKA
461368                OOO GARANT
1608817         MAGNIT MM BOBINA
1601680    MAGNIT GM NOVOCHERKAS
2312451               OOO KORONA
122749            OOO NORSITRANS
2574033      ELEKTRONNAYA SHKOLA
1489534               VETKLINIKA
2310440       MAGNIT MM TOGORIYA
2719319                  KHAGANI
189913                  BLAGRKTS
1741605           MAGAZIN MARIYA
880758           ZELENYJJ OSTROV
Name: tsp_name, dtype: object

In [36]:
tran['txn_comment_1'].sample(20)

2687443     Payment for goods and services
3000285                  Cashless transfer
849450      Payment for goods and services
1005029     Payment for goods and services
77669       Payment for goods and services
2790473     Payment for goods and services
2967359     Payment for goods and services
989160      Payment for goods and services
18152       Cash withdrawal through an ATM
1424632     Payment for goods and services
1403985     Payment for goods and services
86406       Payment for goods and services
2200888     Cash withdrawal through an ATM
2496800     Payment for goods and services
2632831     Payment for goods and services
343637      Payment for goods and services
2188275     Cash withdrawal through an ATM
562360      Payment for goods and services
602394                   Cashless transfer
2039487    Payment by card (bank transfer)
Name: txn_comment_1, dtype: object

In [43]:
# tran[tran['tsp_name'].isna()]

In [45]:
tran['txn_comment_2'].unique()

array([nan, 'Opening Online', 'Purchase payment (web)', '<manual mode>',
       'VISA MONEY TRANSFER', 'MASTERCARD MONEYSEND'], dtype=object)

**Features:**

* tran_amt_rur - amount of transaction
* mcc_cd - category of merchant business type
* tsp_name - transaction direction (who received the money)
* txn_comment_1 - category of transaction
* txn_comment_2 - mostly NaNs, no important info

In [53]:
# Assets Under Management
print(aum.shape)
aum.sample(10)

(117392, 4)


,client_id,month_end_dt,balance_rur_amt,product_code
93832,-1508650601755939053,2019-06-30,270717.0,Current accounts
108851,-7839767071081091401,2019-03-31,-12.0,Broker
49245,-6916069209097360878,2019-04-30,2.0,Current accounts
78988,410876916921664044,2018-12-31,4039.0,Current accounts
63021,-5016855139277913174,2019-08-31,42614.0,Current accounts
77145,-6032965162263481679,2018-12-31,2814.0,Current accounts
47684,-4898849990177348230,2019-07-31,271.0,Current accounts
87535,3349609315336606711,2019-04-30,91599.0,Current accounts
73794,4031854997948142514,2019-05-31,11039.0,Current accounts
86733,2537507496024398334,2019-08-31,325.0,Current accounts


In [55]:
aum.query('client_id == -6032965162263481679').sort_values('month_end_dt')

,client_id,month_end_dt,balance_rur_amt,product_code
77144,-6032965162263481679,2018-11-30,14625.0,Current accounts
77145,-6032965162263481679,2018-12-31,2814.0,Current accounts
77146,-6032965162263481679,2019-01-31,85.0,Current accounts
77143,-6032965162263481679,2019-02-28,907.0,Current accounts
77150,-6032965162263481679,2019-03-31,58.0,Current accounts
77141,-6032965162263481679,2019-04-30,3644.0,Current accounts
77147,-6032965162263481679,2019-05-31,1848.0,Current accounts
77142,-6032965162263481679,2019-06-30,3843.0,Current accounts
77148,-6032965162263481679,2019-07-31,107.0,Current accounts
77149,-6032965162263481679,2019-08-31,3574.0,Current accounts


In [57]:
aum['product_code'].unique()

array(['Current accounts', 'Broker', 'Time account', 'UK', 'NSJ', 'ILI'],
      dtype=object)

**Feature engineering:**

* mean/median/other statistics of client balance

In [68]:
# Client balance
print(balance.shape)
balance.head()

(1194684, 9)


,client_id,crncy_cd,eop_bal_sum_rur,min_bal_sum_rur,max_bal_sum_rur,avg_bal_sum_rur,month_end_dt,prod_cat_name,prod_group_name
0,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
1,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
2,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
3,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand
4,7513301859607023584,810.0,0.0,0.0,0.0,0.0,2018-09-30,CURRENT ACCOUNTS,Cash on demand


In [70]:
balance['prod_cat_name'].unique()

array(['CURRENT ACCOUNTS', 'DEPOSITS', 'Cash on demand',
       'URGENT DEPOSITS BIN', 'SALARY CARDS', 'KN', 'DEBIT CARDS',
       'VIP DEBIT CARDS', 'CASH CREDITS (X-SALE)', 'CREDITS IN CASH OR',
       'LENDING SALARY CLIENTS', 'LOANS FOR EMPLOYEES', nan,
       'CREDIT CARDS', 'VIP CREDIT CARDS', 'PREAD MAP', 'TECHNICAL CARDS',
       'MORTGAGE', 'OTHER MORTGAGE', 'CAR LOANS'], dtype=object)

In [71]:
balance['prod_group_name'].unique()

array(['Cash on demand', 'Time deposits', 'Salary cards', 'Debit cards',
       'PILS', nan, 'Open_card credit card', 'Credit card other', 'POS',
       'Credit card 120 days', 'Prepaid cards', 'Technical cards',
       'Mortgage', 'Car loans'], dtype=object)

In [74]:
balance.query('prod_cat_name == "KN"').prod_group_name.nunique()

8

In [75]:
balance.query('prod_group_name == "Debit cards"').prod_cat_name.nunique()

3

In [78]:
show_nan_number(balance)

NaNs for "client_id": 0
NaNs for "crncy_cd": 888
NaNs for "eop_bal_sum_rur": 0
NaNs for "min_bal_sum_rur": 0
NaNs for "max_bal_sum_rur": 0
NaNs for "avg_bal_sum_rur": 0
NaNs for "month_end_dt": 0
NaNs for "prod_cat_name": 1890
NaNs for "prod_group_name": 96080


In [26]:
# Client features
print(client.shape)
client.head()

(21498, 8)


,client_id,gender,age,region,city,citizenship,education,job_type
0,7513301859607023584,F,33.0,0,115,RUSSIA,NaN,NaN
1,9157009756404187626,F,59.0,17,668,RUSSIA,NaN,NaN
2,-1893104556496814867,M,51.0,28,65,RUSSIA,NaN,NaN
3,6886062013213911831,F,56.0,0,40,RUSSIA,NaN,NaN
4,-8156468515495593794,F,34.0,-1,-1,RUSSIA,HIGHER_PROFESSIONAL,NaN


In [76]:
funnel.shape

(21498, 16)

In [77]:
show_nan_number(client)

NaNs for "client_id": 0
NaNs for "gender": 3
NaNs for "age": 3
NaNs for "region": 0
NaNs for "city": 0
NaNs for "citizenship": 5
NaNs for "education": 12218
NaNs for "job_type": 21423


In [79]:
# previous campaigns for the client
print(campaigns.shape)
campaigns.head()

(113055, 10)


,client_id,agr_flg,otkaz,dumaet,ring_up_flg,not_ring_up_flg,count_comm,month_end_dt,channel,prod
0,7513301859607023584,0,0,0,0,2,2,2018-12-31,CALL,Credit Card
1,7513301859607023584,0,0,0,0,2,2,2019-02-28,CALL,Credit Card
2,-8079817404148237243,0,0,0,2,0,2,2019-08-31,CALL,Credit Card
3,-8079817404148237243,0,0,0,2,0,2,2019-03-31,CALL,Credit Card
4,2897849887287086776,0,0,0,0,16,16,2019-02-28,CALL,Credit Card


In [83]:
campaigns.agr_flg.mean()

0.01736323028614391

In [85]:
campaigns.otkaz.mean()

0.13033479279996463

In [86]:
campaigns.dumaet.mean()

0.014178939454247932

**Feature engineering:**

* Number of previous attempts for each clients

In [28]:
# deals - ??
print(deals.shape)
deals.head()

(109016, 8)


,client_id,agrmnt_start_dt,agrmnt_close_dt,crncy_cd,agrmnt_rate_active,agrmnt_rate_passive,agrmnt_sum_rur,prod_type_name
0,7513301859607023584,2010-08-12,2014-10-30,810.0,NaN,NaN,0.0,Cash on demand
1,7513301859607023584,2013-02-15,2013-08-16,810.0,NaN,NaN,0.0,Cash on demand
2,7513301859607023584,2013-08-16,2014-02-14,810.0,NaN,NaN,0.0,Cash on demand
3,7513301859607023584,2015-07-12,2015-07-12,810.0,NaN,NaN,0.0,Cash on demand
4,7513301859607023584,2015-07-12,2015-07-12,810.0,NaN,NaN,0.0,Cash on demand


In [88]:
show_nan_number(deals)

NaNs for "client_id": 0
NaNs for "agrmnt_start_dt": 0
NaNs for "agrmnt_close_dt": 41789
NaNs for "crncy_cd": 77
NaNs for "agrmnt_rate_active": 78890
NaNs for "agrmnt_rate_passive": 60279
NaNs for "agrmnt_sum_rur": 3601
NaNs for "prod_type_name": 8626


In [29]:
# merchant category mapping to group_name and subgroup_name
print(dict_merchant_category_code.shape)
dict_merchant_category_code.head()

(915, 3)


,mcc_cd,brs_mcc_group,brs_mcc_subgroup
0,3658,Travel,Hotels
1,3659,Travel,Hotels
2,3660,Travel,Hotels
3,3661,Travel,Hotels
4,3662,Travel,Hotels


In [30]:
# client salary + pension payments
print(payments.shape)
payments.head()

(188068, 4)


,client_id,day_dt,sum_rur,pmnts_name
0,-1893104556496814867,2019-06-29,6071.0,Salary receipts
1,-1893104556496814867,2019-08-13,45641.0,Salary receipts
2,-1893104556496814867,2018-10-22,6071.0,Salary receipts
3,-1893104556496814867,2018-09-26,6071.0,Salary receipts
4,-1893104556496814867,2018-09-10,5872.0,Salary receipts


In [31]:
# primary target data and anonimised features
print(funnel.shape)
funnel.head()

(21498, 16)


,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,region_cd
0,7513301859607023584,0,NaN,1,7,13.0,571533.0,15717.0,0.0,0.0,0.0,571852.0,472605.0,10.4,12548.0,86.0
1,9157009756404187626,0,NaN,1,3,13.0,3642369.0,94787.0,0.0,0.0,84823.0,3642369.0,3314257.0,8.9,77210.0,2.0
2,-1893104556496814867,0,NaN,1,5,16.0,352826.0,5500.0,0.0,6822.0,0.0,265893.0,204534.0,8.9,5508.0,52.0
3,6886062013213911831,0,NaN,1,4,3.0,6070615.0,40580.0,0.0,30401.0,0.0,2005731.0,1825051.0,7.9,40583.0,86.0
4,-8156468515495593794,1,138018.05,1,7,14.0,3642369.0,97156.0,81488.0,0.0,160308.0,3642369.0,3314257.0,10.4,78108.0,27.0


In [8]:
# no info about this file was provided yet
print(appl.shape)
appl.head()

(12030, 6)


,client_id,month_end_dt,appl_prod_group_name,appl_prod_type_name,appl_stts_name_dc,appl_sale_channel_name
0,6886062013213911831,2018-10-31,PILS,PILS,The consultation is over,Network
1,4497537671718296975,2018-10-31,PILS,PILS,The consultation is over,Network
2,1316154201527307743,2018-09-30,PILS,PILS,The consultation is over,Network
3,1316154201527307743,2018-09-30,PILS,PILS,The consultation is over,Network
4,1316154201527307743,2018-09-30,PILS,PILS,The consultation is over,Network
